In [ ]:
import asyncio
from bleak import BleakScanner
from playsound import playsound

# Define the address of your phone's BLE device
PHONE_BLE_ADDRESS = 'FC:D4:36:09:F7:XX'

# Custom alarm sound file path
ALARM_SOUND_FILE = r"path to your mp3 alarm file"

def play_alarm():
    print("Playing alarm...")
    playsound(ALARM_SOUND_FILE)  # Use the custom alarm sound file path

def calculate_distance(rssi):
    tx_power = -59  # Typical value, might need calibration for your device
    if rssi == 0:
        return -1.0  # If we cannot determine the distance, return -1
    ratio = rssi * 1.0 / tx_power
    if ratio < 1.0:
        return ratio ** 10
    else:
        return (0.89976) * (ratio ** 7.7095) + 0.111

async def monitor_proximity():
    print("Starting BLE scanner...")
    alarm_count = 0  # Counter to track number of times alarm has played
    while True:
        try:
            print("Scanning for BLE devices...")
            devices = await BleakScanner.discover()
            if not devices:
                print("No devices found.")
            for device in devices:
                device_name = device.name if device.name else "Unknown"
                print(f"Found device: {device_name} ({device.address}) with RSSI: {device.rssi}")
                rssi = device.rssi
                distance = calculate_distance(rssi)
                print(f'RSSI: {rssi}, Estimated Distance: {distance:.2f} meters')
                if distance <= 2.0:
                    print(f"Device {device_name} ({device.address}) is within 2 meters! Triggering alarm...")
                    for _ in range(3):  # Play the alarm 3 times
                        play_alarm()
                        await asyncio.sleep(5)  # Wait between each alarm play
                    print("Alarm played 3 times. Exiting program.")
                    return  # Exit the function and stop scanning after playing the alarm 3 times
                else:
                    print(f"Device {device_name} ({device.address}) is not within 2 meters.")
        except Exception as e:
            print(f"Error during BLE scan: {e}")
        
        await asyncio.sleep(5)  # Check every 5 seconds

# Helper function to run the async function in an existing event loop
def run_asyncio_task(task):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # No running event loop
        loop = None

    if loop and loop.is_running():
        # If the event loop is already running, create a task
        asyncio.create_task(task)
    else:
        # If no event loop is running, run the task
        asyncio.run(task)

if __name__ == '__main__':
    run_asyncio_task(monitor_proximity())
